# Working with overlaying multiple sets of data

In this notebook,  I worked with overlaying data from EPA Flight data detailing power plant locations across Colorado onto OpenStreetMap that detailed the power grid (specifically minor power lines, major power lines, and power cables). 
All of the data here uses geographic coordinate systems - so there may be some inaccuracies, but they should occur at a very small scale. 

In [ ]:
import geopandas as gp
import pandas as pd
import contextily as cx
import osmnx as ox
import matplotlib.pyplot as plt

#OSMnx is used to import OpenStreetMap data about the power grid - specifically the locations of minor lines, major lines, and cables. 
linesData = ox.features.features_from_place("Colorado, USA",tags={'power':[ 'minor_line','line','cable']}, which_result=None, buffer_dist=None) 

In [ ]:
#EPA Flight data is read from the Data folder 
df = pd.read_csv('Data/EpaFlightFinal.csv')
df.head()
#The EPA Flight data is converted into a Geodataframe using geopandas so that it can be graphed. The x and y coordinates are set to the LONGITUDE and LATITUDE columns of the data, respectively.
#The same crs sytem from the OpenStreetMap data is used throughout
gdf = gp.GeoDataFrame(
    df, geometry=gp.points_from_xy(df.LONGITUDE, df.LATITUDE), crs=linesData.crs
)
#The EPA flight data is plotted using geopandas - darker purple colors represents higher CO2 emissions 
gdf.plot(column="GHG QUANTITY (METRIC TONS CO2e)", legend=True, cmap = "RdPu")

In [ ]:
#A buffer of about 2.3 miles is added around the OpenStreetMap data (this is done through the 0.043 - one longitude degree * 0.043 is roughly equal to 2.3 miles). 
#https://www.omnicalculator.com/other/latitude-longitude-distance used to calculate distance
linesBuffers = linesData.buffer(0.043, resolution=16, cap_style='round', join_style='round', mitre_limit=5.0, single_sided=False)
#The buffered data is graphed with a transparency of 0.2
linesBuffers.plot(alpha=0.2)
#The linesBuffers data was in a geoseries, so this line converts it into a Geodataframe which is needed to overlay the data. 
linesBuffersGDF = gp.GeoDataFrame(linesBuffers, geometry=gp.GeoSeries(linesBuffers))

In [ ]:
import matplotlib
#This overlays the EPA Flight data onto the OpenStreetMap buffered data through an intersection, so that only the EPA Flight points that fall inside the buffer zone are returned into doubleData. 
doubleData = linesBuffersGDF.overlay(gdf, how="intersection", keep_geom_type =False)
#The points from DoubleData are plotted on top of the OpenStreetMap lines (which have a transparency of 0.1). Points that are more yellow have higher CO2 emissions, and points that are more red have lower CO2 emissions (logarithmic scale).
linesGraph = linesBuffers.plot(alpha=0.1, figsize=(10, 10))
finalGraph = doubleData.plot(ax=linesGraph, legend = True, legend_kwds={'shrink': 0.3, 'label':'Metric Tons of CO2 Released'}, figsize=(10, 10), 
                             column="GHG QUANTITY (METRIC TONS CO2e)", cmap = "autumn", norm=matplotlib.colors.LogNorm()
                             )
#Creates an underlying Colorado street map
cx.add_basemap(finalGraph, crs=gdf.crs, source=cx.providers.CartoDB.Positron)
finalGraph.set_axis_off()


In [ ]:
inGridPlants = pd.DataFrame(doubleData.drop(columns=['geometry', 0]))
inGridPlants = inGridPlants.drop_duplicates()
inGridEmissions = inGridPlants['GHG QUANTITY (METRIC TONS CO2e)'].sum()
print("Total emissions inside 2.3 miles of the power grid: (Metric Tons of CO2)")
print(inGridEmissions)


totalEmissions = df['GHG QUANTITY (METRIC TONS CO2e)'].sum()
print("Total Colorado CO2 emissions: (Metric Tons of CO2)")
print(totalEmissions)
inGridPlants.head(n=50)

outGridEmissions = totalEmissions - inGridEmissions
print("Total emissions outside of 2.3 miles of the power grid: (Metric Tons of CO2)")
print(outGridEmissions)

print("Percentage of emissions within 2.3 miles of the power grid: ")
print(inGridEmissions/totalEmissions * 100)

In [ ]:
importNational = pd.read_csv('Data/EpaFlightNational.csv')
importNational = importNational.reset_index()
for index, row in importNational.iterrows():
    if ((row['STATE'] == 'VI') or (row['STATE'] == 'GU') or (row['STATE'] == 'PR') or (row['STATE'] == 'AS') or (row['STATE'] == 'MP')):
        importNational = importNational.drop(index)
nationalEmissions = gp.GeoDataFrame(
    importNational, geometry=gp.points_from_xy(importNational.LONGITUDE, importNational.LATITUDE), crs = 'EPSG:4326'
)
nationalEmissions.plot(column="GHG QUANTITY (METRIC TONS CO2e)", legend=True, cmap = "autumn")

In [ ]:
nationalPowerLines = gp.read_file('Data/Electric_Power_Transmission_Lines.geojson')
nationalLinesBuffers = nationalPowerLines.buffer(0.043, resolution=16, cap_style='round', join_style='round', mitre_limit=5.0, single_sided=False)
nationalLinesBuffers.to_crs('EPSG:4326')
nationalBuffersGDF = gp.GeoDataFrame(nationalLinesBuffers, geometry=gp.GeoSeries(nationalLinesBuffers))
nationalBuffersGDF.plot()

In [ ]:
import matplotlib
overlayNational = nationalBuffersGDF.overlay(nationalEmissions, how="symmetric_difference", keep_geom_type =False)

nationalLinesGraph = nationalBuffersGDF.plot(alpha=1, figsize=(20, 20))
nationalLinesGraph.set_xlim(-175, -50)
finalNationalGraph = overlayNational.plot(ax=nationalLinesGraph, legend = True, legend_kwds={'shrink': 0.3, 'label':'Metric Tons of CO2 Released'}, figsize=(20, 20), 
                             column="GHG QUANTITY (METRIC TONS CO2e)", cmap = "autumn", norm=matplotlib.colors.LogNorm()
                             )
cx.add_basemap(finalNationalGraph, crs='EPSG:4326', source=cx.providers.CartoDB.Positron)
finalNationalGraph.set_axis_off()

In [ ]:
nationalInGridSources = pd.DataFrame(overlayNational.drop(columns=['geometry', 0]))
nationalInGridSources = nationalInGridSources.drop_duplicates()
inGridEmissions = nationalInGridSources['GHG QUANTITY (METRIC TONS CO2e)'].sum()
print("Total emissions inside 2.3 miles of the power grid: (Metric Tons of CO2)")
print(inGridEmissions)

totalEmissions = nationalEmissions['GHG QUANTITY (METRIC TONS CO2e)'].sum()
print("Total national CO2 emissions: (Metric Tons of CO2)")
print(totalEmissions)

outGridEmissions = totalEmissions - inGridEmissions
print("Total emissions outside of 2.3 miles of the power grid: (Metric Tons of CO2)")
print(outGridEmissions)

print("Percentage of emissions within 2.3 miles of the power grid: ")
print(inGridEmissions/totalEmissions * 100)
print('\n')

totalEmissionSources = len(nationalEmissions)
powerEmissionsInGrid = len(nationalInGridSources)
print("Total emission sources: ")
print(totalEmissionSources)
print("Total emission sources within 2.3 miles of the grid: ")
print(powerEmissionsInGrid)
print("Percentage of emission sources within 2.3 miles of the grid")
print(powerEmissionsInGrid / totalEmissionSources * 100)


In [ ]:
from numpy import log10
for index, row in overlayNational.iterrows():
    if (row['GHG QUANTITY (METRIC TONS CO2e)'] <= 0):
        overlayNational = overlayNational.drop(index)

overlayNational['log(Metric Tons CO2)'] = log10(overlayNational['GHG QUANTITY (METRIC TONS CO2e)'])
overlayNational.head(n=20)

In [ ]:
#Creates an interactive map showing 


m = nationalBuffersGDF.explore(tooltip = False, figsize=(10, 10), legend=False)
overlayNational.explore(m=m, column='log(Metric Tons CO2)', figsize=(10, 10), cmap = 'autumn', legend=True, tooltip="GHG QUANTITY (METRIC TONS CO2e)")